<a href="https://colab.research.google.com/github/klinsc/sdd-nonggof-v1/blob/main/2_Llama3_x_GGUF_RAG_LlamaIndex_pdf_colab_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p>
    <img src="https://cdn-uploads.huggingface.co/production/uploads/6424f01ea4f3051f54dbbd85/oqVQ04b5KiGt5WOWJmYt8.png" alt="LlamaIndex" width="100" height="100">
    <img src="https://cdn4.iconfinder.com/data/icons/file-extensions-1/64/pdfs-512.png" alt="PDF" width="100" height="100">
    <img src="https://huggingface.co/front/assets/huggingface_logo-noborder.svg" alt="PDF" width="100" height="100">
</p>

# LlamaIndex
## [Youtube Video covering this notebook](https://youtu.be/QNKeNiRjtGQ?si=UFH9WL5nXvmj2EM1)
- [LlamaIndex Website](https://www.llamaindex.ai/)
- [LangChain Website](https://python.langchain.com/docs/get_started/introduction)
-[Retrieval-Augmented Generation (RAG)](https://research.ibm.com/blog/retrieval-augmented-generation-RAG)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install libraries

In [ ]:
%%capture
!pip install llama-index
!pip install llama-index-llms-llama-cpp
!pip install llama-index-embeddings-huggingface

!pip install llama-index-postprocessor-flag-embedding-reranker
!pip install git+https://github.com/FlagOpen/FlagEmbedding.git

In [ ]:
!pip list

Package                                           Version
------------------------------------------------- -------------------
absl-py                                           1.4.0
accelerate                                        1.7.0
aiohappyeyeballs                                  2.6.1
aiohttp                                           3.11.15
aiosignal                                         1.3.2
aiosqlite                                         0.21.0
alabaster                                         1.0.0
albucore                                          0.0.24
albumentations                                    2.0.7
ale-py                                            0.11.0
altair                                            5.5.0
annotated-types                                   0.7.0
antlr4-python3-runtime                            4.9.3
anyio                                             4.9.0
argon2-cffi                                       23.1.0
argon2-cffi-bindings      

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python==0.3.7 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu125 --upgrade --force-reinstall --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu125
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 66.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 266.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 218.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 331.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 338.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 231.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=41082200 sha256=76ab6a3c2877cca81216684e5ca04020522db1388cd0c0252bf447bcd8032db7
  Stored in directory: /tmp/pip-ephem-wheel-cache-9_qa51

In [ ]:
#!pip install numpy==1.26.4
!pip install numpy==1.26.4 --force-reinstall --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 336.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


## Implementation

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from IPython.display import Markdown, display

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


## Load documents and build index

In [ ]:
%%time
documents = SimpleDirectoryReader("/content/drive/MyDrive/Courses/RAG/SDD").load_data()
len(documents)

CPU times: user 11min 35s, sys: 6.1 s, total: 11min 41s
Wall time: 11min 40s


3039

In [ ]:
documents[0].metadata

{'page_label': '1',
 'file_name': '537 Guide for Transformer Fire Safety Practices.pdf',
 'file_path': '/content/drive/MyDrive/Courses/RAG/SDD/537 Guide for Transformer Fire Safety Practices.pdf',
 'file_type': 'application/pdf',
 'file_size': 4172940,
 'creation_date': '2025-05-28',
 'last_modified_date': '2018-02-14'}

In [ ]:
documents[100]

Document(id_='0b517d5d-fc66-40c7-bc0b-df769f891e67', embedding=None, metadata={'page_label': '101', 'file_name': '537 Guide for Transformer Fire Safety Practices.pdf', 'file_path': '/content/drive/MyDrive/Courses/RAG/SDD/537 Guide for Transformer Fire Safety Practices.pdf', 'file_type': 'application/pdf', 'file_size': 4172940, 'creation_date': '2025-05-28', 'last_modified_date': '2018-02-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Guide for Transformer Fire Safety Practices\n92\nFigure 42 illustrates how a fire barrier is used to protect two adjacent transformers. FM Global\nadvises that the fire wall should ex tend at least 600m

### Llama3.1 8B 4bit GGUF ( 4.92 GB. )

https://huggingface.co/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF

In [ ]:
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
          prompt += f"<|start_header_id|>system<|end_header_id|>{message.content}<|eot_id|>\n"
          #prompt += f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>{message.content}<|eot_id|>\n"
        elif message.role == 'user':
          prompt += f"<|start_header_id|>user<|end_header_id|>{message.content}<|eot_id|>\n"
        elif message.role == 'assistant':
          prompt += f"<|start_header_id|>assistant<|end_header_id|>{message.content}<|eot_id|>\n"

    # ensure we start with a system prompt, insert blank if needed
    # if not prompt.startswith("<|begin_of_text|><|start_header_id|>system"):
    #     prompt = "<|begin_of_text|>" + prompt

    # add final assistant prompt
    prompt = prompt + "<|start_header_id|>assistant<|end_header_id|><|eot_id|>\n"
    return prompt

def completion_to_prompt(completion):
    return f"<|start_header_id|>system<|end_header_id|><|eot_id|>\n<|start_header_id|>user<|end_header_id|>{completion}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n"
    #return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|><|eot_id|>\n<|start_header_id|>user<|end_header_id|>{completion}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n"

In [ ]:
# ลิงก์จาก repo ใน huggingface
model_url = "https://huggingface.co/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf"
#model_url = 'https://huggingface.co/lmstudio-community/Llama-3.2-3B-Instruct-GGUF/resolve/main/Llama-3.2-3B-Instruct-Q8_0.gguf'
#model_url = 'https://huggingface.co/lmstudio-community/Llama-3.2-3B-Instruct-GGUF/resolve/main/Llama-3.2-3B-Instruct-Q4_K_M.gguf'

# From Google Drive
#model_path = '/content/drive/MyDrive/BaseModel/Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf'

from llama_index.llms.llama_cpp import LlamaCPP

llm = LlamaCPP(
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    context_window=8192,
    max_new_tokens=1024, # อาจจะตอบเพิ่มออกมาอีก 1024 bytes
    generate_kwargs={},
    model_kwargs={
        "repetition-penalty":1.4,
        "no_repeat_ngram_size": 4,
         #"response_format": { "type": "json_object" },
        "n_gpu_layers": -1
        },
    #model_kwargs={"n_gpu_layers": 33},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA L4, compute capability 8.9, VMM: yes
llama_model_load_from_file_impl: using device CUDA0 (NVIDIA L4) - 22503 MiB free
llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from /tmp/llama_index/models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                        

In [ ]:
llm

LlamaCPP(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7efebbd01710>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7efebce63ec0>, completion_to_prompt=<function completion_to_prompt at 0x7efebce60180>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model_url='https://huggingface.co/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf', model_path='/tmp/llama_index/models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf', temperature=0.1, max_new_tokens=1024, context_window=8192, generate_kwargs={'temperature': 0.1, 'max_tokens': 1024}, model_kwargs={'n_ctx': 8192, 'verbose': True, 'repetition-penalty': 1.4, 'no_repeat_ngram_size': 4, 'n_gpu_layers': -1}, verbose=True)

## Inference Model ( without RAG )

In [ ]:
# %%time
# llm.generate_kwargs={'temperature': 0.1, 'max_tokens': 512}

# response = llm.complete("กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ" )
# display(Markdown(f"{response.text}"))
# len(response.text)

In [ ]:
%%time
llm.generate_kwargs={'temperature': 0.1, 'max_tokens': 1024}
stream_response = llm.stream_complete(
    "กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ"
)
for t in stream_response:
    print(t.delta, end="")
print()

โยคะ เป็นศิลปะการฝึกตนและฝึกสมาธิเพื่อสุขภาพและความสมดุลทางกายและจิต โดยมีหลายประเภท เช่น

1. กามโยคะ (Hatha Yoga) - ศิลปะการฝึกตนเพื่อสุขภาพและความสมดุลทางกาย โดยใช้เทคนิคการยืดเหยียดและการหายใจลึก

2. ภวโยคะ (Bhakti Yoga) - ศิลปะการฝึกตนเพื่อความเชื่อมั่นและความภักดี โดยใช้เทคนิคการฝึกสมาธิและการสักการะ

3. ทิฏฐิโยคะ (Jnana Yoga) - ศิลปะการฝึกตนเพื่อความเข้าใจและความเห็นใจ โดยใช้เทคนิคการฝึกสมาธิและการสังเกต

4. อวิชชาโยคะ (Kundalini Yoga) - ศิลปะการฝึกตนเพื่อการปลดปล่อยพลังภายใน โดยใช้เทคนิคการฝึกสมาธิและการหายใจลึก

5. ราชนโยคะ (Raja Yoga) - ศิลปะการฝึกตนเพื่อการปลดปล่อยจิตวิญญาณ โดยใช้เทคนิคการฝึกสมาธิและการสังเกต

6. คาร์นโยคะ (Karma Yoga) - ศิลปะการฝึกตนเพื่อการปลดปล่อยจิตวิญญาณ โดยใช้เทคนิคการฝึกสมาธิและการทำความดี

7. นิมิตโยคะ (Mantra Yoga) - ศิลปะการฝึกตนเพื่อการปลดปล่อยจิตวิญญาณ โดยใช้เทคนิคการฝึกสมาธิและการใช้คำสอน

8. ลัทธิโยคะ (Laya Yoga) - ศิลปะการฝึกตนเพื่อการปลดปล่อยจิตวิญญาณ โดยใช้เทคนิคการฝึกสมาธิและการหายใจลึก

9. นาธโยคะ (Natha Yoga) - ศิลปะการฝึกตนเพื่อการปลดปล่อ

llama_perf_context_print:        load time =     212.22 ms
llama_perf_context_print: prompt eval time =     211.89 ms /    41 tokens (    5.17 ms per token,   193.50 tokens per second)
llama_perf_context_print:        eval time =   12351.23 ms /   570 runs   (   21.67 ms per token,    46.15 tokens per second)
llama_perf_context_print:       total time =   13539.87 ms /   611 tokens


ังเกต
CPU times: user 13.5 s, sys: 116 ms, total: 13.6 s
Wall time: 13.5 s


In [ ]:
torch.cuda.empty_cache()

### Stream Response

In [ ]:
# %%time
# llm.generate_kwargs={'temperature': 0.1, 'max_tokens': 1024}
# stream_response = llm.stream_complete(
#     "กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ"
# )
# for t in stream_response:
#     print(t.delta, end="" , flush=True)
# print()

In [ ]:
# %%time
# from llama_index.core.llms import ChatMessage

# messages = [
#     ChatMessage(role="system", content="You are an AI assistant that answers questions in Thai language"),
#     ChatMessage(role="user", content="กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ"),
# ]
# stream_response = llm.stream_chat(messages)

# for t in stream_response:
#     print(t.delta, end="" , flush=True)
# print()

## Embedding Model

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")                      # size 8192
#embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large")  # size 1024

# local file
#embed_model = HuggingFaceEmbedding(model_name='/content/drive/MyDrive/EmbeddingModel/bge-m3')

In [ ]:
from llama_index.core import Settings

Settings.chunk_size    = 3840
Settings.chunk_overlap = 256
Settings.llm = llm
Settings.embed_model   = embed_model

## Indexing

In [ ]:
%%time
index = VectorStoreIndex.from_documents(documents ,show_progress=True)

Parsing nodes:   0%|          | 0/3039 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/991 [00:00<?, ?it/s]

CPU times: user 4min 27s, sys: 874 ms, total: 4min 28s
Wall time: 4min 19s


## Query
Start querying by getting the default query engine
https://docs.llamaindex.ai/en/stable/module_guides/querying/

In [ ]:
# from llama_index.core import PromptTemplate

# template = (
#     "Context information is below.\n"
#     "---------------------\n"
#     "{context_str}\n"
#     "---------------------\n"
#     "Given the context information and not prior knowledge, "
#     "answer the query in Thai language.\n"
#     "Query: {query_str}\n"
#     "Answer: "
# )
# prompt_template = PromptTemplate(template)

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=20,
    verbose=True,
    streaming=True,
    #text_qa_template=prompt_template,
)

response = query_engine.query("วิธีป้องกันฟ้าผ่า")
response.print_response_stream()

# display(Markdown(f"{response}"))
# clear_gpu_memory()

Llama.generate: 9 prefix-match hit, remaining 7071 prompt tokens to eval
llama_perf_context_print:        load time =    4419.93 ms
llama_perf_context_print: prompt eval time =    4890.47 ms /  7071 tokens (    0.69 ms per token,  1445.87 tokens per second)
llama_perf_context_print:        eval time =   11333.41 ms /   412 runs   (   27.51 ms per token,    36.35 tokens per second)
llama_perf_context_print:       total time =   16892.22 ms /  7483 tokens
Llama.generate: 9 prefix-match hit, remaining 6905 prompt tokens to eval
llama_perf_context_print:        load time =    4419.93 ms
llama_perf_context_print: prompt eval time =    4770.25 ms /  6905 tokens (    0.69 ms per token,  1447.51 tokens per second)
llama_perf_context_print:        eval time =   23645.02 ms /   858 runs   (   27.56 ms per token,    36.29 tokens per second)
llama_perf_context_print:       total time =   30195.64 ms /  7763 tokens
Llama.generate: 33 prefix-match hit, remaining 2221 prompt tokens to eval
llama_perf

Based on the provided context, the refined answer to the query "วิธีป้องกันฟ้าผ่า" (How to prevent lightning strikes) is:

การป้องกันฟ้าผ่าสามารถทำได้ดังนี้

1. ใช้ระบบป้องกันฟ้าผ่า เช่น ระบบป้องกันฟ้าผ่าแบบแอร์เทอร์มินัล (air terminal) หรือ ระบบป้องกันฟ้าผ่าแบบมาสต์ (mast)
2. ให้แน่ใจว่าโครงสร้างมีการป้องกันฟ้าผ่าอย่างดี
3. พิจารณาประเภทของการก่อสร้าง, ความถี่และความรุนแรงของพายุฟ้าคะนอง, ค่าของอาคารและทรัพย์สิน, ความเสี่ยงต่อผู้คน, ความอ่อนไหวต่อฟ้าผ่า, และเศรษฐศาสตร์เมื่อตัดสินใจว่าจะมีการป้องกันฟ้าผ่าหรือไม่
4. ใช้ระบบป้องกันฟ้าผ่าแบบแมทเทอร์ (metallic) หรือ ระบบป้องกันฟ้าผ่าแบบฟาราเดย์ (Faraday) เพื่อป้องกันการเสียหายจากฟ้าผ่า
5. ดูแลและทดสอบระบบป้องกันฟ้าผ่าอย่างสม่ำเสมอ

นอกจากนี้ยังสามารถใช้ระบบป้องกันฟ้าผ่าแบบอื่นๆ เช่น ระบบป้องกันฟ้าผ่าแบบคาเทนารี (catenary system) หรือ ระบบป้องกันฟ้าผ่าแบบพายเรือ (shipboard lightning protection system) เพื่อป้องกันการเสียหายจากฟ้าผ่า

การป้องกันฟ้าผ่าอย่างมีประสิทธิภาพต้องอาศัยการวางแผนและออกแบบระบบป้องกันฟ้าผ่าอย่างดี รวมถึงการดูแลและทดสอบร

llama_perf_context_print:        load time =    4419.93 ms
llama_perf_context_print: prompt eval time =    4103.33 ms /  6222 tokens (    0.66 ms per token,  1516.33 tokens per second)
llama_perf_context_print:        eval time =   14799.77 ms /   548 runs   (   27.01 ms per token,    37.03 tokens per second)
llama_perf_context_print:       total time =   20272.95 ms /  6770 tokens


In [ ]:
# Print the number of source nodes
num_source_nodes = len(response.source_nodes)
print(f"Number of source nodes: {num_source_nodes}")

# Loop over source nodes and print meta data
for s in response.source_nodes:
    print(f"Node Score: {s.score}")
    print(s.node.metadata)

Number of source nodes: 2
Node Score: 0.623294117668057
{'page_label': '10', 'file_name': 'พระไตรปิฎก_21อังคุตรนิกาย-จตุกกนิบาต.pdf', 'file_path': '/content/drive/MyDrive/Courses/RAG/C2_Llama3x-RAG-Workshop/dataset/Tri45-5files/พระไตรปิฎก_21อังคุตรนิกาย-จตุกกนิบาต.pdf', 'file_type': 'application/pdf', 'file_size': 1112869, 'creation_date': '2024-12-12', 'last_modified_date': '2024-07-07'}
Node Score: 0.5962243557236705
{'page_label': '11', 'file_name': 'พระไตรปิฎก_21อังคุตรนิกาย-จตุกกนิบาต.pdf', 'file_path': '/content/drive/MyDrive/Courses/RAG/C2_Llama3x-RAG-Workshop/dataset/Tri45-5files/พระไตรปิฎก_21อังคุตรนิกาย-จตุกกนิบาต.pdf', 'file_type': 'application/pdf', 'file_size': 1112869, 'creation_date': '2024-12-12', 'last_modified_date': '2024-07-07'}


In [ ]:
response.response_txt

'กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ เป็นโยคะ 4 ประการ'

In [ ]:
response

StreamingResponse(response_gen=<generator object stream_completion_response_to_tokens.<locals>.gen at 0x7d3e9f095e50>, source_nodes=[NodeWithScore(node=TextNode(id_='9157334f-e405-4cf5-ae51-dad504191b28', embedding=None, metadata={'page_label': '10', 'file_name': 'พระไตรปิฎก_21อังคุตรนิกาย-จตุกกนิบาต.pdf', 'file_path': '/content/drive/MyDrive/Courses/RAG/C2_Llama3x-RAG-Workshop/dataset/Tri45-5files/พระไตรปิฎก_21อังคุตรนิกาย-จตุกกนิบาต.pdf', 'file_type': 'application/pdf', 'file_size': 1112869, 'creation_date': '2024-12-12', 'last_modified_date': '2024-07-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='fbbab046-4bb1-4348-a189-fab23435ef1a', node_type='4', metadata={'page_label': '10', 'file_name': 'พระ

## Storing

In [ ]:
index.storage_context.persist(persist_dir="./storage")

In [ ]:
cp -r ./storage /content/drive/MyDrive/Courses/RAG/storage

# Loading the Index

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./storage")
#storage_context = StorageContext.from_defaults(persist_dir="/content/drive/MyDrive/TrainModel/storage_tri45")

In [ ]:
llm

LlamaCPP(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ad8c655e990>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7ad8c5f3c900>, completion_to_prompt=<function completion_to_prompt at 0x7ad8c5f3c860>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model_url='https://huggingface.co/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf', model_path='/tmp/llama_index/models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf', temperature=0.1, max_new_tokens=1024, context_window=8192, generate_kwargs={'temperature': 0.1, 'max_tokens': 1024}, model_kwargs={'n_ctx': 8192, 'verbose': True, 'repetition-penalty': 1.4, 'no_repeat_ngram_size': 4, 'n_gpu_layers': -1}, verbose=True)

In [ ]:
embed_model

HuggingFaceEmbedding(model_name='BAAI/bge-m3', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ad654597510>, num_workers=None, max_length=8192, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None, show_progress_bar=False)

In [ ]:
from llama_index.core import Settings
# Settings.chunk_size    = 3840
# Settings.chunk_overlap = 256
Settings.llm         = llm
Settings.embed_model = embed_model

In [ ]:
load_index = load_index_from_storage(storage_context=storage_context)
load_index

### **Query as Retriever in Vector Database only**

Node Postprocessor   
https://docs.llamaindex.ai/en/stable/module_guides/querying/node_postprocessors/

In [ ]:
retrieval_engine = load_index.as_retriever(similarity_top_k=2)
response         = retrieval_engine.retrieve("กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ")
display(Markdown(f"{response[0]}"))
display(Markdown(f"{response[1]}"))

Node ID: 9157334f-e405-4cf5-ae51-dad504191b28
Text: พระสุตตันตปฎก เลม ๑๓ อังคุตรนิกาย จตุกกนิบาต - หนาที่ 10
โทษ และอุบายเครื่องสลัดออกแหงทิฐิทั้งหลาย ตามความเปนจริง
ความกําหนัดเพราะทิฐิ   ความเพลิดเพลินเพราะทิฐิความเยื่อใยเพราะทิฐิ
ความหมกมุนเพราะทิฐิ ความกระหายเพราะ  ทิฐิ ความเรารอนเพราะทิฐิ
ความหยั่งลงเพราะทิฐิ และความทะยานอยากเพราะทิฐิ ในทิฐิ  ทั้งหลาย
ยอมเกิดขึ้น นี้เราเรียกวาทิฏ...
Score:  0.623


Node ID: 04447e70-06cc-442e-9754-ac54014d783f
Text: พระสุตตันตปฎก เลม ๑๓ อังคุตรนิกาย จตุกกนิบาต - หนาที่ 11
ทั้งหลาย บุคคลบางคนในโลกนี้ ยอมรูชัดซึ่งความเกิด ความดับ คุณ
โทษและอุบายเครื่องสลัด  ออกแหงทิฐิทั้งหลาย ตามความเปนจริง
เมื่อเขารูชัดซึ่งความเกิด ความดับ คุณ โทษ และ
อุบายเครื่องสลัดออกแหงทิฐิทั้งหลายตามความเปนจริง
ความกําหนัดเพราะทิฐิ ความเพลิดเพลิน  เพราะทิฐิ คว ามเย ื่อใยเพรา...
Score:  0.596


### Query with LLM
Response Mode
https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes/

In [ ]:
torch.cuda.empty_cache()

In [ ]:
%%time
llm.generate_kwargs={'temperature': 0.1, 'max_tokens': 512}

query_engine = load_index.as_query_engine(
    similarity_top_k=4 ,
    streaming=True)
response     = query_engine.query("กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ")
response.print_response_stream()
#display(Markdown(f"{response}"))

llama_perf_context_print:        load time =    2039.96 ms
llama_perf_context_print: prompt eval time =    2039.23 ms /  3667 tokens (    0.56 ms per token,  1798.23 tokens per second)
llama_perf_context_print:        eval time =     859.58 ms /    35 runs   (   24.56 ms per token,    40.72 tokens per second)
llama_perf_context_print:       total time =    2945.59 ms /  3702 tokens
Llama.generate: 9 prefix-match hit, remaining 1302 prompt tokens to eval


กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ

llama_perf_context_print:        load time =    2039.96 ms
llama_perf_context_print: prompt eval time =     589.02 ms /  1302 tokens (    0.45 ms per token,  2210.45 tokens per second)
llama_perf_context_print:        eval time =     593.28 ms /    26 runs   (   22.82 ms per token,    43.82 tokens per second)
llama_perf_context_print:       total time =    1231.54 ms /  1328 tokens


CPU times: user 4.11 s, sys: 283 ms, total: 4.39 s
Wall time: 4.36 s


### Chat Engine

usage pattern guide.   
https://docs.llamaindex.ai/en/stable/module_guides/deploying/chat_engines/usage_pattern/

In [ ]:
chat_engine = load_index.as_chat_engine(
    llm=llm,
    similarity_top_k=1,
    chat_mode="condense_question",
    streaming=True,
    verbose=True,
)
#chat_engine.reset()

In [ ]:
collected_messages = []

streaming_response = chat_engine.stream_chat("กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ")
for token in streaming_response.response_gen:
    print(token, end="")
    collected_messages.append(token)

full_message = ''.join(collected_messages)
display(Markdown(f"{full_message}"))

Querying with: กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ


Llama.generate: 9 prefix-match hit, remaining 1376 prompt tokens to eval


กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ เป

llama_perf_context_print:        load time =    2039.96 ms
llama_perf_context_print: prompt eval time =     613.05 ms /  1376 tokens (    0.45 ms per token,  2244.51 tokens per second)
llama_perf_context_print:        eval time =     797.93 ms /    35 runs   (   22.80 ms per token,    43.86 tokens per second)
llama_perf_context_print:       total time =    1462.63 ms /  1411 tokens


็นโยคะ 4 ประการ

กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ เป็นโยคะ 4 ประการ

## Rerank

In [ ]:
from llama_index.postprocessor.flag_embedding_reranker import (
    FlagEmbeddingReranker,
)
rerank = FlagEmbeddingReranker(model="BAAI/bge-reranker-v2-m3", top_n=5)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
%%time
query_engine = load_index.as_query_engine(
    similarity_top_k=10,
    streaming=True,
    node_postprocessors=[rerank]
)

response = query_engine.query(
    "กามโยคะ ภวโยคะ ทิฏฐิโยคะ และอวิชชาโยคะ",
)
response.print_response_stream()

## RAG Summarization Tool

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core import SimpleDirectoryReader

# load documents
#documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()
documents = SimpleDirectoryReader(
    input_files=["/content/drive/MyDrive/Dataset/Tri45-5files/พระไตรปิฎก_21อังคุตรนิกาย-จตุกกนิบาต.pdf"]).load_data()
len(documents)

In [ ]:
documents = documents[:10]

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=3840 ,chunk_overlap=256)
nodes    = splitter.get_nodes_from_documents(documents ,show_progress=True)

In [ ]:
from llama_index.core import SummaryIndex

summary_index  = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
    verbose=False,
    streaming=True,
)

In [ ]:
%%time
llm.generate_kwargs={'temperature': 0.8, 'max_tokens': 2048}

response = summary_query_engine.query(
    "สรุปเอกสาร เป็นแต่ละหัวข้อ พร้อมอธิบายเนื้อหาและยกตัวอย่างประกอบ"
)
response.print_response_stream()

In [ ]:
num_source_nodes = len(response.source_nodes)
print(f"Number of source nodes: {num_source_nodes}")

# Loop over source nodes and print meta data
for s in response.source_nodes:
    print(f"Node Score: {s.score}")
    print(s.node.metadata)

## Advanced RAG (Routing, Sub-Questions)

In [ ]:
# from llama_index.core.tools import QueryEngineTool, ToolMetadata

# vector_tool = QueryEngineTool(
#     index.as_query_engine(),
#     metadata=ToolMetadata(
#         name="vector_search",
#         description="Useful for searching for specific facts.",
#     ),
# )

# summary_tool = QueryEngineTool(
#     index.as_query_engine(response_mode="tree_summarize"),
#     metadata=ToolMetadata(
#         name="summary",
#         description="Useful for summarizing an entire document.",
#     ),
# )

In [ ]:
# from llama_index.core.query_engine import RouterQueryEngine

# query_engine = RouterQueryEngine.from_defaults(
#     [vector_tool, summary_tool],
#     select_multi=False,
#     verbose=True
# )

# response = query_engine.query(
#     "Tell me about the song meet the grahams - why is it significant"
# )

# display(Markdown(f"{response}"))
# print(len(str(response)))